# 🎬 UVG MAX – Premium AI Video Generator

**Automated cinematic video creation powered by AI**

### Features:
- 🎙️ Azure TTS with word-level timing
- 🎥 Pexels/Pixabay stock footage
- ✨ Cinematic effects (LUT, grain, motion)
- 📝 Animated captions
- 🎵 Audio mastering & SFX

### API Keys Required:
- [Gemini API](https://makersuite.google.com/app/apikey) (script generation)
- [Pexels API](https://www.pexels.com/api/) (stock footage)
- [Azure TTS](https://azure.microsoft.com/en-us/products/ai-services/text-to-speech) (voice synthesis)

---
**GitHub**: [github.com/shya-bubu/uvg-max-engine](https://github.com/shya-bubu/uvg-max-engine)

## 📦 Step 1: Setup

In [ ]:
# Clone repository
!git clone https://github.com/shya-bubu/uvg-max-engine.git
%cd uvg-max-engine

# Install dependencies
!pip install -q -r requirements.txt

# Verify FFmpeg
!ffmpeg -version | head -1

print("✅ Setup complete!")

## 🔑 Step 2: Configure API Keys

In [ ]:
import os
from getpass import getpass

print("🔐 Enter your API keys:")
print("-" * 40)

# Required
os.environ["GEMINI_API_KEY"] = getpass("Gemini API Key: ")
os.environ["PEXELS_KEY"] = getpass("Pexels API Key: ")

# Azure TTS (optional - uses mock if empty)
azure_key = getpass("Azure TTS Key (or press Enter for mock): ")
if azure_key:
    os.environ["AZURE_SPEECH_KEY"] = azure_key
    os.environ["AZURE_SPEECH_REGION"] = input("Azure Region (e.g., eastus): ") or "eastus"
    os.environ["UVG_MOCK_MODE"] = "false"
else:
    os.environ["UVG_MOCK_MODE"] = "true"
    print("ℹ️ Using mock TTS mode")

# Optional
pixabay = getpass("Pixabay Key (optional): ")
if pixabay:
    os.environ["PIXABAY_KEY"] = pixabay

print("\n✅ API keys configured!")

## 🎬 Step 3: Generate Video

In [ ]:
from uvg_core.uvg_pipeline import UVGPipeline, PipelineConfig
from IPython.display import display, Video, clear_output
import time

# ============================================
# 🎨 CUSTOMIZE YOUR VIDEO
# ============================================
TOPIC = "The beauty of nature and wildlife"  # @param {type:"string"}
STYLE = "cinematic"  # @param ["cinematic", "tiktok", "motivational", "corporate", "travel"]
PLATFORM = "youtube"  # @param ["youtube", "tiktok", "instagram", "twitter"]
# ============================================

print(f"🎬 Topic: {TOPIC}")
print(f"🎨 Style: {STYLE}")
print(f"📱 Platform: {PLATFORM}")
print("-" * 50)

# Progress display
def show_progress(step, pct):
    bar = "█" * int(pct * 20) + "░" * (20 - int(pct * 20))
    print(f"\r[{bar}] {pct:.0%} {step}          ", end="")

# Configure pipeline
config = PipelineConfig(
    style_pack=STYLE,
    target_platform=PLATFORM,
    mock_mode=os.environ.get("UVG_MOCK_MODE", "true").lower() == "true"
)

# Run pipeline
pipeline = UVGPipeline(config)
result = pipeline.run(
    topic=TOPIC,
    output_name="my_video",
    on_progress=show_progress
)

print("\n")
if result.success:
    print("✅ VIDEO GENERATED!")
    print(f"📁 Output: {result.output_path}")
    print(f"⏱️ Duration: {result.duration_sec:.1f}s")
    print(f"🎬 Scenes: {result.scenes_processed}")
    if result.warnings:
        print(f"⚠️ Warnings: {len(result.warnings)}")
else:
    print("❌ Generation failed")
    for e in result.errors:
        print(f"   Error: {e}")

## 🎥 Step 4: Watch Video

In [ ]:
from IPython.display import Video, display
import glob
import os

# Find video
video_paths = [
    "uvg_output/export/*.mp4",
    "uvg_output/final/*.mp4",
    "uvg_output/**/*.mp4"
]

video = None
for pattern in video_paths:
    matches = glob.glob(pattern, recursive=True)
    if matches:
        video = matches[0]
        break

if video and os.path.exists(video):
    print(f"🎬 Playing: {video}")
    display(Video(video, embed=True, width=400))
else:
    print("⚠️ No video found. Check generation logs.")

## 🖼️ Step 5: View Thumbnail

In [ ]:
from PIL import Image
from IPython.display import display
import glob

thumbs = glob.glob("uvg_output/thumbnails/*.jpg") + glob.glob("uvg_output/thumbnails/*.png")

if thumbs:
    print("🖼️ Thumbnail:")
    img = Image.open(thumbs[0])
    display(img)
else:
    print("ℹ️ No thumbnail generated")

## 💾 Step 6: Download

In [ ]:
from google.colab import files
import glob

# Find files to download
videos = glob.glob("uvg_output/**/*.mp4", recursive=True)
thumbs = glob.glob("uvg_output/thumbnails/*", recursive=True)

print("📥 Available downloads:")
all_files = videos + thumbs
for f in all_files[:5]:
    print(f"  - {f}")

if videos:
    print(f"\n⬇️ Downloading: {videos[0]}")
    files.download(videos[0])

---

## 📝 Advanced: Custom Script

Use a custom script instead of AI generation:

In [ ]:
# Custom script example
custom_script = {
    "title": "My Custom Video",
    "scenes": [
        {
            "text": "Welcome to my amazing video about technology.",
            "duration": 5.0,
            "emotion": "excited",
            "visual_cue": "futuristic technology, neon lights"
        },
        {
            "text": "Innovation is changing the world every day.",
            "duration": 4.0,
            "emotion": "inspiring",
            "visual_cue": "city skyline, modern architecture"
        },
        {
            "text": "The future is now. Thanks for watching!",
            "duration": 4.0,
            "emotion": "hopeful",
            "visual_cue": "sunrise, nature"
        }
    ]
}

# Run with custom script
from uvg_core.uvg_pipeline import UVGPipeline, PipelineConfig

pipeline = UVGPipeline(PipelineConfig(style_pack="cinematic"))
result = pipeline.run(script=custom_script, output_name="custom_video")

print(f"Result: {'✅ Success' if result.success else '❌ Failed'}")
print(f"Output: {result.output_path}")

---

## ⚙️ Configuration Options

| Option | Values | Description |
|--------|--------|-------------|
| `style_pack` | cinematic, tiktok, motivational, corporate, travel, romantic, tech | Visual style preset |
| `target_platform` | youtube, tiktok, instagram, twitter | Export optimization |
| `enable_captions` | true/false | Animated captions |
| `enable_sfx` | true/false | Sound effects |
| `mock_mode` | true/false | Use mock APIs for testing |

---

**Star ⭐ the repo**: [github.com/shya-bubu/uvg-max-engine](https://github.com/shya-bubu/uvg-max-engine)